# 2020_GSE26320_ENCODE

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

from IPython.display import display
import pandas as pd

import seaborn as sns
sns.set_style("whitegrid")
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import scipy as sp
import os, re
import glob
from tqdm.auto import tqdm
import subprocess, tempfile

import plotly.graph_objects as go
import plotly.express as px

In [ ]:
# Single track per modification per cell
CELLS = [
    'GM12878',
    'H1',
    'HepG2',
    'HMEC',
    'HSMM',
    'Huvec',
    'K562',
    'NHEK',
    'NHLF',
]

MODIFICATIONS = ['H3K27me3', 'H3K27ac', 'H3K4me3', 'H3K4me1', 'H3K36me3']

# MACS2

In [ ]:
MACS2_FOLDER='/mnt/stripe/shpynov/2020_GSE26320/macs2'
MACS2_LEVELS = ['q0.05', 'broad0.05']

dfm = pd.DataFrame(columns=['gsm', 'cell', 'modification', 'level', 'file', 'peaks', 'length'], dtype=object)
for file in tqdm(glob.glob(MACS2_FOLDER + '/*.*Peak')):
    if 'gapped' in file:
        continue
    level = next((l for l in MACS2_LEVELS if f'_{l}' in file), None)
    modification = next((m for m in MODIFICATIONS if f'_{m}' in file), None)
    cell = next((c for c in CELLS if f'_{c}' in file), None)
#     print(file, level, modification, cell)
    if level and modification and cell:
        gsm = re.sub('_.*', '', os.path.basename(file))
        out = ! awk '{{N+=1;L+=($$3-$$2)}} END{{print(N,L)}}' {file}
        if out[0].strip() != '':
            peaks, length = out[0].split(' ') 
        else:
            peaks, length = 0, 0
        dfm.loc[len(dfm)] = (gsm, cell, modification, f'macs2 {level}', file, peaks, length)
        
# Fix types
dfm['peaks'] = dfm['peaks'].astype(int)
dfm['length'] = dfm['length'].astype(int)

In [ ]:
len(dfm)

In [ ]:
dfm['f'] = dfm['modification']
dfm_mean = dfm.groupby(['f', 'level'])['peaks'].mean().reset_index().sort_values(by=['f', 'level'])
dfm_std = dfm.groupby(['f', 'level'])['peaks'].std().reset_index().fillna(0).sort_values(by=['f', 'level'])

fig = go.Figure(layout=go.Layout(title=go.layout.Title(text="Peaks")))
for l in sorted(set(dfm_mean['level'])):
    fig.add_trace(go.Scatter(x=dfm_mean.loc[dfm_mean['level']==l]['f'], 
                             y=dfm_mean.loc[dfm_mean['level']==l]['peaks'], 
                             name=f"{l} mean", line_shape='linear'))
    fig.add_trace(go.Scatter(x=dfm_std.loc[dfm_std['level']==l]['f'], 
                             y=dfm_std.loc[dfm_std['level']==l]['peaks'], 
                             name=f"{l} std", line_shape='linear', 
                             line=dict(dash='dot')))
fig.show()

In [ ]:
macs2levels2process = set(['macs2 q0.05', 'macs2 broad0.05'])

# SICER

In [ ]:
SICER_FOLDER='/mnt/stripe/shpynov/2020_GSE26320/sicer'
SICER_LEVELS = ['FDR0.05']

dfsc = pd.DataFrame(columns=['gsm', 'cell', 'modification', 'level', 'file', 'peaks', 'length'], dtype=object)
for file in tqdm(glob.glob(SICER_FOLDER + '/*islands-summary*')):
    level = next((l for l in SICER_LEVELS if f'-{l}' in file), None)
    modification = next((m for m in MODIFICATIONS if f'_{m}' in file), None)
    cell = next((c for c in CELLS if f'_{c}' in file), None)
#     print(file, level, modification, cell)
    if level and modification and cell:
        gsm = re.sub('_.*', '', os.path.basename(file))
        out = ! awk '{{N+=1;L+=($$3-$$2)}} END{{print(N,L)}}' {file}
        if out[0].strip() != '':
            peaks, length = out[0].split(' ') 
        else:
            peaks, length = 0, 0
        dfsc.loc[len(dfsc)] = (gsm, cell, modification, f'sicer {level}', file, peaks, length)

# Fix types
dfsc['peaks'] = dfsc['peaks'].astype(int)
dfsc['length'] = dfsc['length'].astype(int)

In [ ]:
len(dfsc)

In [ ]:
dfsc['f'] = dfsc['modification']
dfsc_mean = dfsc.groupby(['f', 'level'])['peaks'].mean().reset_index().sort_values(by=['f', 'level'])
dfsc_std = dfsc.groupby(['f', 'level'])['peaks'].std().reset_index().fillna(0).sort_values(by=['f', 'level'])

fig = go.Figure(layout=go.Layout(title=go.layout.Title(text="Peaks")))
for l in sorted(set(dfsc_mean['level'])):
    fig.add_trace(go.Scatter(x=dfsc_mean.loc[dfsc_mean['level']==l]['f'], 
                             y=dfsc_mean.loc[dfsc_mean['level']==l]['peaks'], 
                             name=f"{l} mean", line_shape='linear'))
    fig.add_trace(go.Scatter(x=dfsc_std.loc[dfsc_std['level']==l]['f'], 
                             y=dfsc_std.loc[dfsc_std['level']==l]['peaks'], 
                             name=f"{l} std", line_shape='linear', 
                             line=dict(dash='dot')))
fig.show()

In [ ]:
sicerlevels2process = set(['sicer FDR0.05'])

# SPAN

In [ ]:
SPAN_FOLDER='/mnt/stripe/shpynov/2020_GSE26320/span'
GAPS = [5]
FDRS = ['0.05']
# SPAN_LEVELS = ['200_1E-6_5', '200_0.01_5']
SPAN_LEVELS = []
for gap in GAPS:
    for fdr in FDRS:
        SPAN_LEVELS.append(f'200_{fdr}_{gap}')

dfs = pd.DataFrame(columns=['gsm', 'cell', 'modification', 'level', 'file', 'peaks', 'length'], dtype=object)
for file in tqdm(glob.glob(SPAN_FOLDER + '/*.peak')):
    if 'Input' in file:
        continue
    level = next((l for l in SPAN_LEVELS if f'_{l}' in file), None)
    modification = next((m for m in MODIFICATIONS if f'_{m}' in file), None)
    cell = next((c for c in CELLS if f'_{c}' in file), None)
#     print(file, level, modification, cell)
    if level and modification and cell:
        gsm = re.sub('_.*', '', os.path.basename(file))
        out = ! awk '{{N+=1;L+=($$3-$$2)}} END{{print(N,L)}}' {file}
        if out[0].strip() != '':
            peaks, length = out[0].split(' ') 
        else:
            peaks, length = 0, 0
        dfs.loc[len(dfs)] = (gsm, cell, modification, f'span {level}', file, peaks, length)
        
# Fix types
dfs['peaks'] = dfs['peaks'].astype(int)
dfs['length'] = dfs['length'].astype(int)

In [ ]:
len(dfs)

In [ ]:
import re
dfs['fdr'] = [float(re.sub('span 200_|_(0|5|10)', '', l)) for l in dfs['level']]
dfs['gap'] = [int(re.sub('.*_', '', l)) for l in dfs['level']]
dfs.sort_values(by=['fdr', 'gap'], inplace=True)
dfs.head()

In [ ]:
dfs['f'] = dfs['modification']
dfs_mean = dfs.groupby(['f', 'level'])['peaks'].mean().reset_index().sort_values(by=['f', 'level'])
dfs_std = dfs.groupby(['f', 'level'])['peaks'].std().reset_index().fillna(0).sort_values(by=['f', 'level'])

fig = go.Figure(layout=go.Layout(title=go.layout.Title(text="Peaks")))
for l in sorted(set(dfs_mean['level'])):
    fig.add_trace(go.Scatter(x=dfs_mean.loc[dfs_mean['level']==l]['f'], 
                             y=dfs_mean.loc[dfs_mean['level']==l]['peaks'], 
                             name=f"{l} mean", line_shape='linear'))
    fig.add_trace(go.Scatter(x=dfs_std.loc[dfs_std['level']==l]['f'], 
                             y=dfs_std.loc[dfs_std['level']==l]['peaks'], 
                             name=f"{l} std", line_shape='linear', 
                             line=dict(dash='dot')))
fig.show()

In [ ]:
# Plot peaks number versus FDR for different modifications and GAPs
for m in sorted(set(dfs['modification'])):
    t = dfs.loc[dfs['modification'] == m]
    for gap in sorted(set(dfs['gap'])):
        t2 = t.loc[t['gap']==gap]
        fig = go.Figure()
        for cell in set(t2['cell']):
            t3 = t2.loc[t2['cell'] == cell]
            fig.add_trace(go.Scatter(x=np.log10(t3["fdr"]), y=t3["peaks"], 
                                     mode='lines+markers',
                                     name=cell))
        fig.update_xaxes(title='log10 fdr')
        fig.update_yaxes(title=f'{m} gap {gap} peaks')
        fig.show()

In [ ]:
spanlevels2process = set(['span 200_0.05_5'])

# Summary

In [ ]:
dfa = pd.concat([dfm.loc[[l in macs2levels2process for l in dfm['level']]],
                 dfsc.loc[[l in sicerlevels2process for l in dfsc['level']]],
                 dfs.loc[[l in spanlevels2process for l in dfs['level']]]])

In [ ]:
dfa['f'] = dfa['modification']
dfa_mean = dfa.groupby(['f', 'level'])['peaks'].mean().reset_index().sort_values(by=['f', 'level'])
dfa_std = dfa.groupby(['f', 'level'])['peaks'].std().reset_index().fillna(0).sort_values(by=['f', 'level'])

fig = go.Figure(layout=go.Layout(title=go.layout.Title(text="Peaks")))
for l in sorted(set(dfa_mean['level'])):
    fig.add_trace(go.Scatter(x=dfa_mean.loc[dfa_mean['level']==l]['f'], 
                             y=dfa_mean.loc[dfa_mean['level']==l]['peaks'], 
                             name=f"{l} mean", line_shape='linear'))
    fig.add_trace(go.Scatter(x=dfa_std.loc[dfa_std['level']==l]['f'], 
                             y=dfa_std.loc[dfa_std['level']==l]['peaks'], 
                             name=f"{l} std", line_shape='linear', 
                             line=dict(dash='dot')))
fig.show()

In [ ]:
dfa['avg_length'] = dfa['length'] / dfa['peaks']
dfa.loc[~np.isfinite(dfa["avg_length"]), "avg_length"] = 0.0

In [ ]:
# # List file to create session
# for m in MODIFICATIONS:
#     for c in CELLS:
#         bw = glob.glob(f'/mnt/stripe/shpynov/2020_roadmapepigenomics/bams_bws/*{c}.{m}.*.bw')[0]
#         print(bw)
#         dfcm = dfa.loc[np.logical_and(dfa['cell']==c, dfa['modification']==m)]
#         for l in sorted(set(dfa['level'])):
#             peaks = list(dfcm.loc[dfcm['level'] == l]['file'])
#             if peaks:
#                 peaks = peaks[0]
#                 print(f'{os.path.dirname(peaks)}/bb/{os.path.basename(peaks)}.bb')

# Group analysis

In [ ]:
def plot_data_cells(df, cid, value, description):
    cids = sorted(set(df[cid]))
    axs = {}
    total = len(cids) * 5
    fig = plt.figure(figsize=(int(total * .75), 4))
    offset = 0
    for m in MODIFICATIONS:
        data = df.loc[df['modification'] == m].sort_values(by=[cid])
        xlabels = []
        for c in data[cid]:
            if c not in xlabels:
                xlabels.append(c)
        w = len(cids)
        ax = plt.subplot2grid((1, total), (0, offset), colspan=w)

        sns.barplot(data=data, 
                     x=cid, y=value,
                     capsize=.2, errwidth=2,
                     edgecolor="black",
                     ax = ax)

        sns.swarmplot(data=data,
                      x=cid, y=value,
                      size=1,
                      color="black",
                      alpha=0.5,
                      ax = ax)
        ax.legend().set_visible(False)
        axs[ax] = plt.ylim()
        if offset > 0:
            ax.get_yaxis().set_ticklabels([])
            ax.set_ylabel('')
        else:
            ax.set_ylabel(description)
        
        offset += w
        ax.set_xlabel('')
        ax.set_title(m)
        plt.xticks(range(0, len(xlabels)), xlabels, rotation=45)
            
    ymin = np.min([v[0] for v in axs.values()])
    ymax = np.max([v[1] for v in axs.values()])
 
    for ax in axs.keys():
        ax.set_ylim(bottom = ymin, top = ymax)
    plt.tight_layout()

In [ ]:
plot_data_cells(dfa, 'level', 'peaks', 'Peaks')
plt.show()
plot_data_cells(dfa, 'level', 'avg_length', 'Average peak length')
plt.show()

In [ ]:
dfa['f'] = dfa['level']
plot_data_cells(dfa, 'f', 'peaks', 'Peaks')
plt.show()
plot_data_cells(dfa, 'f', 'avg_length', 'Average peak length')
plt.show()

# Consistency analysis

In [ ]:
import tempfile

def bedl(file):
    try:
        tf = pd.read_csv(file, sep='\t', header=None)
        return (tf[2] - tf[1]).sum()
    except:
        return 0. # Empty file

tools_comparison = pd.DataFrame(columns=['gsm', 'modification', 'tool1', 'tool2', 
                                         'peaks1', 'peaks2', 'length1', 'length2', 
                                         'overlap1', 'overlap2', 'intersect'], dtype=object)

tools = list(sorted(set(dfa['level'])))
for gsm in tqdm(sorted(set(dfa['gsm']))):
    for i in range(len(tools)):
        for j in range(i + 1, len(tools)):
            t1 = dfa.loc[(dfa['gsm'] == gsm) & (dfa['level'] == tools[i])]
            t2 = dfa.loc[(dfa['gsm'] == gsm) & (dfa['level'] == tools[j])]
            m = t1['modification'].values[0]
            file1 = t1['file'].values[0]
            peaks1 = t1['peaks'].values[0]
            length1 = bedl(file1)
            file2 = t2['file'].values[0]
            peaks2 = t2['peaks'].values[0]
            length2 = bedl(file2)
            overlap1 = !bedtools intersect -a {file1} -b {file2} -wa -u | wc -l
            overlap1 = int(overlap1[0])
            overlap2 = !bedtools intersect -b {file1} -a {file2} -wa -u | wc -l            
            overlap2 = int(overlap2[0])            
            tf = tempfile.mktemp() 
            !bedtools intersect -b {file1} -a {file2} > {tf}
            intersectionl = bedl(tf)
            tools_comparison.loc[len(tools_comparison)] = \
                (gsm, m, tools[i], tools[j], peaks1, peaks2, length1, length2, 
                 overlap1, overlap2, intersectionl)
            
tools_comparison

In [ ]:
to = pd.DataFrame(columns=['Modification', 'GSM', 'Comparison', 'Overlap'], dtype=object)
for _, row in tools_comparison.iterrows():
    m = row['modification']
    gsm = row['gsm']
    t1 = row['tool1']
    t2 = row['tool2']
    peaks1 = row['peaks1']
    peaks2 = row['peaks2']
    overlap1 = row['overlap1']
    overlap2 = row['overlap2']    
    to.loc[len(to)] = (m, gsm, f'Comparison {t1} vs {t2}', overlap1 / peaks1 if peaks1 !=0 else 0)
    to.loc[len(to)] = (m, gsm, f'Comparison {t2} vs {t1}', overlap2 / peaks2 if peaks2 !=0 else 0)

display(to) 

plt.figure(figsize=(18, 4))
sns.barplot(data=to.loc[['span' in c for c in to['Comparison']]], 
            x='Modification', y='Overlap', hue='Comparison', capsize=.1, errwidth=2, edgecolor="black")
plt.show()

In [ ]:
tj = pd.DataFrame(columns=['Modification', 'GSM', 'Comparison', 'Jaccard'], dtype=object)
for _, row in tools_comparison.iterrows():
    m = row['modification']
    gsm = row['gsm']
    t1 = row['tool1']
    t2 = row['tool2']
    length1 = row['length1']
    length2 = row['length2']
    intersection = row['intersect']
    try:
        jaccard = intersection / (length1 + length2 - intersection)
    except:
        jaccard = 0
    tj.loc[len(tj)] = (m, gsm, f'Comparison {t1} vs {t2}', jaccard)

display(tj) 

plt.figure(figsize=(18, 4))
sns.barplot(data=tj.loc[['span' in c for c in tj['Comparison']]], 
            x='Modification', y='Jaccard', hue='Comparison', capsize=.1, errwidth=2, edgecolor="black")
plt.show()

# Coverage analysis
```
# Experimental signal-to-noise ratio and peak calling contrast
for M in $(ls /mnt/stripe/shpynov/2020_GSE26320/span/fit/*.span); do echo $M; java -cp experiments/build/libs/experiments-dev.jar -Dconfig.path=/home/user/.epigenome/config.properties org.jetbrains.bio.experiments.SPANPeakCallingContrastExperiment $M; done
```

In [ ]:
from io import StringIO

ts = []
for info in tqdm(glob.glob('/mnt/stripe/bio/experiments/span_peak_calling_contrast/*.info')):
    name = os.path.basename(info)
    gsm = re.findall('(GSM[0-9]+)', name)[0]
    with open(info) as t:
        info = ''.join(t.readlines())
        if 'Error' in info:
            continue
#         print(info)
        sn = float(re.findall('Signal to noise: ([0-9\.]+)', info)[0])
#         print('SN', sn)
        totalscore = int(re.findall('Total Score: ([0-9]+)', info)[0])
#         print('TS', totalscore)
        dft = re.sub('(.|\n)+Total Score:[^\n]+\n', '', info)
        t = pd.read_csv(StringIO(dft), sep='\t')
        t['GSM'] = gsm
        t['SNR'] = sn
        t['TOTAL_SCORE'] = totalscore
        ts.append(t)
t = pd.concat(ts)

t['PEAKS_RPKM'] = t['PEAKS_SCORE'] / (t['PEAKS_LENGTH'] / 1000) / (t['TOTAL_SCORE'] / 1000000)
t['SHORES_RPKM'] = t['SHORES_SCORE'] / (t['SHORES_LENGTH'] / 1000) / (t['TOTAL_SCORE'] / 1000000)
t['CONTRAST'] = t['PEAKS_RPKM'] / t['SHORES_RPKM']
infodf = t
infodf = infodf.loc[infodf['FDR'].astype(float)>=1e-10]
infodf.head()

In [ ]:
t = pd.merge(left=infodf, left_on='GSM', right=dfs[['gsm', 'cell', 'modification']].drop_duplicates(), right_on='gsm')
infodf = t
infodf.head()

In [ ]:
import matplotlib

def rgb2hex(r, g, b):
    r, g, b = r * 255, g * 255, b * 255
    return "#{0:02x}{1:02x}{2:02x}".format(int(r), int(g), int(b))

cells = list(set(infodf['cell']))
cmap = matplotlib.cm.get_cmap('tab20', len(cells))
cell_colors = dict(zip(cells, [rgb2hex(*cmap(i)[:3]) for i in range(len(cells))]))
cell_colors

In [ ]:
# Peaks number vs FDR
for m in set(infodf['modification']):
    tm = infodf.loc[infodf['modification'] == m]
    fig = go.Figure()
    
    for g in sorted(set(tm['GAP'])):
        tmg = tm.loc[tm['GAP'] == g]
        for c in sorted(set(tmg['cell'])):
            tmgc = tmg.loc[tmg['cell']==c].copy()
            tmgc.sort_values(by=['FDR'], inplace=True)
            if g == 0:
                fig.add_trace(go.Scatter(x=np.log10(tmgc['FDR']), y=tmgc['PEAKS_NUMBER'], 
                         mode='lines',
                         name=f'{c} {g}',
                         line = dict(color=cell_colors[c], width=2, dash='dash')))
            elif g == 5:
                fig.add_trace(go.Scatter(x=np.log10(tmgc['FDR']), y=tmgc['PEAKS_NUMBER'], 
                         mode='lines',
                         name=f'{c} {g}',
                         line = dict(color=cell_colors[c], width=2)))
            elif g == 10:
                fig.add_trace(go.Scatter(x=np.log10(tmgc['FDR']), y=tmgc['PEAKS_NUMBER'], 
                         mode='lines',
                         name=f'{c} {g}',
                         line = dict(color=cell_colors[c], width=2, dash='dot')))

    fig.update_xaxes(title=f'{m} Log10 FDR')
    fig.update_yaxes(title=f'{m} Peaks')
    fig.show()

In [ ]:
infodf['PEAKS_AVG_LENGTH'] = infodf['PEAKS_LENGTH'] / infodf['PEAKS_NUMBER']
# Peaks number vs Peaks average length
for m in set(infodf['modification']):
    tm = infodf.loc[infodf['modification'] == m]
    fig = go.Figure()
    
    for g in sorted(set(tm['GAP'])):
        tmg = tm.loc[tm['GAP'] == g]
        for c in sorted(set(tmg['cell'])):
            tmgc = tmg.loc[tmg['cell']==c].copy()
            tmgc.sort_values(by=['FDR'], inplace=True)
            if g == 0:
                fig.add_trace(go.Scatter(x=tmgc['PEAKS_NUMBER'], y=tmgc['PEAKS_AVG_LENGTH'], 
                         mode='lines',
                         name=f'{c} {g}',
                         line = dict(color=cell_colors[c], width=2, dash='dash')))
            elif g == 5:
                fig.add_trace(go.Scatter(x=tmgc['PEAKS_NUMBER'], y=tmgc['PEAKS_AVG_LENGTH'], 
                         mode='lines',
                         name=f'{c} {g}',
                         line = dict(color=cell_colors[c], width=2)))
            elif g == 10:
                fig.add_trace(go.Scatter(x=tmgc['PEAKS_NUMBER'], y=tmgc['PEAKS_AVG_LENGTH'], 
                         mode='lines',
                         name=f'{c} {g}',
                         line = dict(color=cell_colors[c], width=2, dash='dot')))

    fig.update_xaxes(title=f'{m} Peaks number')
    fig.update_yaxes(title=f'{m} Peaks average length')
    fig.show()

In [ ]:
# # Contrast vs FDR
# for m in set(infodf['modification']):
#     tm = infodf.loc[infodf['modification'] == m]
#     for g in sorted(set(tm['GAP'])):
#         tmg = tm.loc[tm['GAP'] == g]
#         fig = go.Figure()
#         for c in sorted(set(tmg['cell'])):
#             tmgc = tmg.loc[tmg['cell']==c].copy()
#             tmgc.sort_values(by=['FDR'], inplace=True)
#             fig.add_trace(go.Scatter(x=np.log10(tmgc['FDR']), y=tmgc['CONTRAST'], 
#                                      mode='lines+markers',
#                                      name=c))
#         fig.update_xaxes(title=f'{m} Log10 FDR')
#         fig.update_yaxes(title=f'{m} gap {g} Contrast')
#         fig.show()

In [ ]:
# Contrast vs Peaks RPKM
for m in set(infodf['modification']):
    tm = infodf.loc[infodf['modification'] == m]
    display(tm.loc[np.logical_and(t['GAP']==5, t['FDR'].isin([0.1, 1e-6]))][
        ['modification', 'cell', 'GAP', 'FDR', 'SNR', 'PEAKS_NUMBER', 'PEAKS_RPKM', 'CONTRAST']
    ].sort_values(by=['cell']))
    for g in sorted(set(tm['GAP'])):
        tmg = tm.loc[tm['GAP'] == g]
        fig = go.Figure()
        for c in sorted(set(tmg['cell'])):
            tmgc = tmg.loc[tmg['cell']==c].copy()
            tmgc.sort_values(by=['FDR'], inplace=True)
            fig.add_trace(go.Scatter(x=np.log10(tmgc['PEAKS_RPKM']), y=tmgc['CONTRAST'], 
                                     mode='lines+markers',
                                     name=c))
        fig.update_xaxes(title=f'{m} Peaks log10 RPKM')
        fig.update_yaxes(title=f'{m} gap {g} Contrast')
        fig.show()

In [ ]:
# Contrast vs Peaks RPKM
for m in set(infodf['modification']):
    tm = infodf.loc[infodf['modification'] == m]
    fig = go.Figure()
    display(tm.loc[np.logical_and(t['GAP']==5, t['FDR'].isin([0.1, 1e-6]))][
        ['modification', 'cell', 'GAP', 'FDR', 'SNR', 'PEAKS_NUMBER', 'PEAKS_RPKM', 'CONTRAST']
    ].sort_values(by=['cell']))
    
    for g in sorted(set(tm['GAP'])):
        tmg = tm.loc[tm['GAP'] == g]
        for c in sorted(set(tmg['cell'])):
            tmgc = tmg.loc[tmg['cell']==c].copy()
            tmgc.sort_values(by=['FDR'], inplace=True)
            if g == 0:
                fig.add_trace(go.Scatter(x=np.log10(tmgc['PEAKS_RPKM']), y=tmgc['CONTRAST'], 
                         mode='lines',
                         name=f'{c} {g}',
                         line = dict(color=cell_colors[c], width=2, dash='dash')))
            elif g == 5:
                fig.add_trace(go.Scatter(x=np.log10(tmgc['PEAKS_RPKM']), y=tmgc['CONTRAST'], 
                         mode='lines',
                         name=f'{c} {g}',
                         line = dict(color=cell_colors[c], width=2)))
            elif g == 10:
                fig.add_trace(go.Scatter(x=np.log10(tmgc['PEAKS_RPKM']), y=tmgc['CONTRAST'], 
                         mode='lines',
                         name=f'{c} {g}',
                         line = dict(color=cell_colors[c], width=2, dash='dot')))

    fig.update_xaxes(title=f'{m} Peaks log10 RPKM')
    fig.update_yaxes(title=f'{m} Contrast')
    fig.show()

In [ ]:
# # Contrast vs Peaks number
# for m in set(infodf['modification']):
#     tm = infodf.loc[infodf['modification'] == m]
#     display(tm.loc[np.logical_and(t['GAP']==5, t['FDR']==0.1)][
#         ['modification', 'cell', 'SNR', 'PEAKS_NUMBER', 'PEAKS_RPKM', 'CONTRAST']
#     ].sort_values(by=['cell']))
#     for g in sorted(set(tm['GAP'])):
#         tmg = tm.loc[tm['GAP'] == g]
#         fig = go.Figure()
#         for c in sorted(set(tmg['cell'])):
#             tmgc = tmg.loc[tmg['cell']==c].copy()
#             tmgc.sort_values(by=['FDR'], inplace=True)
#             fig.add_trace(go.Scatter(x=np.log10(tmgc['PEAKS_NUMBER']), y=tmgc['CONTRAST'], 
#                                      mode='lines+markers',
#                                      name=c))
#         fig.update_xaxes(title=f'{m} Peaks log10 number')
#         fig.update_yaxes(title=f'{m} gap {g} Contrast')
#         fig.show()

In [ ]:
# Real signal-to-noise ratio
infodf['REAL_SNR'] = (infodf['PEAKS_SCORE'] / infodf['PEAKS_LENGTH']) / (infodf['TOTAL_SCORE'] - infodf['PEAKS_SCORE']) * (3*10e9 - infodf['PEAKS_LENGTH'])
infodf.head()

In [ ]:
# FDR vs REAL signal to noise
for m in set(infodf['modification']):
    tm = infodf.loc[infodf['modification'] == m]
    fig = go.Figure()
    
    for g in sorted(set(tm['GAP'])):
        tmg = tm.loc[tm['GAP'] == g]
        for c in sorted(set(tmg['cell'])):
            tmgc = tmg.loc[tmg['cell']==c].copy()
            tmgc.sort_values(by=['FDR'], inplace=True)
            if g == 0:
                fig.add_trace(go.Scatter(x=np.log10(tmgc['FDR']), y=np.log10(tmgc['REAL_SNR']), 
                         mode='lines+markers',
                         name=f'{c} {g}',
                         line = dict(color=cell_colors[c], width=1, dash='dash')))
            elif g == 5:
                fig.add_trace(go.Scatter(x=np.log10(tmgc['FDR']), y=np.log10(tmgc['REAL_SNR']), 
                                         mode='lines+markers',
                                         name=f'{c} {g}',
                                         line = dict(color=cell_colors[c], width=1)))
            elif g == 10:
                fig.add_trace(go.Scatter(x=np.log10(tmgc['FDR']), y=np.log10(tmgc['REAL_SNR']), 
                         mode='lines+markers',
                         name=f'{c} {g}',
                         line = dict(color=cell_colors[c], width=1, dash='dot')))

    fig.update_xaxes(title='Log10 FDR')
    fig.update_yaxes(title=f'{m} gap {g} Log10 SNR')
    fig.show()

In [ ]:
# PEAKS NUMBER vs REAL signal to noise
for m in set(infodf['modification']):
    tm = infodf.loc[infodf['modification'] == m]
    fig = go.Figure()
    
    for g in sorted(set(tm['GAP'])):
        tmg = tm.loc[tm['GAP'] == g]
        for c in sorted(set(tmg['cell'])):
            tmgc = tmg.loc[tmg['cell']==c].copy()
            tmgc.sort_values(by=['FDR'], inplace=True)
            if g == 0:
                fig.add_trace(go.Scatter(x=np.log10(tmgc['PEAKS_NUMBER']), y=np.log10(tmgc['REAL_SNR']), 
                         mode='lines+markers',
                         name=f'{c} {g}',
                         line = dict(color=cell_colors[c], width=1, dash='dash')))
            elif g == 5:
                fig.add_trace(go.Scatter(x=np.log10(tmgc['PEAKS_NUMBER']), y=np.log10(tmgc['REAL_SNR']), 
                                         mode='lines+markers',
                                         name=f'{c} {g}',
                                         line = dict(color=cell_colors[c], width=1)))
            elif g == 10:
                fig.add_trace(go.Scatter(x=np.log10(tmgc['PEAKS_NUMBER']), y=np.log10(tmgc['REAL_SNR']), 
                         mode='lines+markers',
                         name=f'{c} {g}',
                         line = dict(color=cell_colors[c], width=1, dash='dot')))

    fig.update_xaxes(title='Log10 Peaks number')
    fig.update_yaxes(title=f'{m} gap {g} Log10 SNR')
    fig.show()